In [3]:
import numpy as np
import pandas as pd
import pickle
import os.path
import math
import sys
import csv
import matplotlib.pyplot as plt
from sklearn.externals import joblib
import scipy.integrate as spi
import pylab as pl

## Load data

In [12]:
n_bytes = 2**31
max_bytes = 2**31 - 1

def load_pickle(file_path):
    bytes_in = bytearray(0)
    input_size = os.path.getsize(file_path)
    with open(file_path, 'rb') as f_in:
        for _ in range(0, input_size, max_bytes):
            bytes_in += f_in.read(max_bytes)
        return pickle.loads(bytes_in)
path = "/Users/maimai/Desktop/project/May/data/givenchy/pickle"  
users = load_pickle(path+"/users.dat")
friends = load_pickle(path+"/friends.dat")
network = load_pickle(path+"/network.dat")
old_data = load_pickle(path+"/30_data(0).dat")
data = load_pickle(path+"/30_data(4).dat")

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55559 entries, 0 to 3321
Data columns (total 89 columns):
label                                         55559 non-null int64
UsM_deltaDays                                 55559 non-null int64
UsM_statusesCount                             55559 non-null int64
UsM_followersCount                            55559 non-null int64
UsM_favouritesCount                           55559 non-null int64
UsM_friendsCount                              55559 non-null int64
UsM_listedCount                               55559 non-null int64
UsM_normalizedUserStatusesCount               55559 non-null float64
UsM_normalizedUserFollowersCount              55559 non-null float64
UsM_normalizedUserFavouritesCount             55559 non-null float64
UsM_normalizedUserListedCount                 55559 non-null float64
UsM_normalizedUserFriendsCount                55559 non-null float64
UsM_deltaDays0                                55559 non-null int64
UsM_statuse

# infec_candidates 拓扑需要扩展更新！！

In [10]:
######## infec_candidates ########
temp=[]
infec = [temp for i in range(5011)]
users["infec_candidates"]= None
record_num = int(network.describe().iloc[0,0])

for i in range(record_num):
    source_candidates = users.loc[i,"source_candidates"]
    #users.loc[i,"infec_candidates"] = list()
    for j in range(len(source_candidates)):
        candidates = source_candidates[j]
        if infec[candidates] == []:
            infec[candidates]=[i]
        else :
            infec[candidates]+=[i]

users.loc[:,"infec_candidates"]=infec

users.head()

,user,user_id,time_lapsed,followers_count,friends_count,user_created_days,user_statuses_count,user_listed_count,user_favourites_count,normalized_user_statuses_count,...,normalized_user_favourites_count,normalized_user_listed_count,normalized_user_friends_count,mention_and_reply,source_candidates,source_index,seed_index,generation,time_since_seed,infec_candidates
0,davelackie,100766356,0.00,143119,4643,2993,181393,555,11938,60.605747,...,3.988640,0.185433,1.551286,[],"[3, 5, 6, 7, 10, 11, 14, 15, 19, 20, 24, 74, 8...",None,0,0,0,"[1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1..."
1,alexanderskhan,3186545203,0.15,753,3428,1042,12853,8,34186,12.334933,...,32.808061,0.007678,3.289827,[0],[0],0,0,1,0.15,"[14, 4334]"
2,consiglierela,4134992843,0.23,2161,396,856,46934,36,78797,54.829439,...,92.052570,0.042056,0.462617,[0],"[0, 12, 21, 32, 48, 748, 819, 972, 2358, 2778,...",0,0,1,0.23,"[12, 32, 818, 1635, 2358, 4699]"
3,ImpactPrincess,1955472014,0.63,242,642,1613,23179,55,28995,14.370118,...,17.975821,0.034098,0.398016,[0],"[0, 115, 577]",0,0,1,0.63,"[0, 3587]"
4,Damek0Masca,406301175,1.13,116,239,2318,2407,12,1058,1.038395,...,0.456428,0.005177,0.103106,[],[],None,4,0,0,[]


# input: network + seeds + time t

## ①. Network

In [4]:
network_simulation = pd.DataFrame(columns=["t_current","user","user_id","source_candidates","infec_candidates",
                                           "followers_count","friends_count","source_id","seed_user_id",
                                           "SIR","generations","time_lapsed","time_since_seed"])
network_simulation["user"] = users["user"]
network_simulation["user_id"] = users["user_id"]
network_simulation["source_candidates"] = users["source_candidates"]
network_simulation["infec_candidates"] = users["infec_candidates"]
network_simulation["followers_count"] = users["followers_count"]
network_simulation["friends_count"] = users["friends_count"]
network_simulation.head()

,t_current,user,user_id,source_candidates,infec_candidates,followers_count,friends_count,source_id,seed_user_id,SIR,generations,time_lapsed,time_since_seed
0,NaN,davelackie,100766356,"[3, 5, 6, 7, 10, 11, 14, 15, 19, 20, 24, 74, 8...","[1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...",143119,4643,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,alexanderskhan,3186545203,[0],"[14, 4334]",753,3428,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,consiglierela,4134992843,"[0, 12, 21, 32, 48, 748, 819, 972, 2358, 2778,...","[12, 32, 818, 1635, 2358, 4699]",2161,396,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,ImpactPrincess,1955472014,"[0, 115, 577]","[0, 3587]",242,642,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Damek0Masca,406301175,[],[],116,239,NaN,NaN,NaN,NaN,NaN,NaN


## ②. Seeds

In [6]:
######## All Seeds 99 ########
seeds = network.copy()

record_num_network = network.describe().iloc[0,0]
record_num_network = int(record_num_network)
for i in range(record_num_network):
    record = network.loc[i,"generation"]
    if record != 0:
        seeds.drop([i],axis=0, inplace = True)
seeds.head()

,user,user_id,source_id,time_lapsed,followers_count,friends_count,seed_user_id,generation,time_since_seed
0,davelackie,100766356,NaN,0.00,143119,4643,1.007664e+08,0.0,0.0
4,Damek0Masca,406301175,NaN,1.13,116,239,4.063012e+08,0.0,0.0
21,Fashionista_com,15933910,NaN,17.00,2254291,825,1.593391e+07,0.0,0.0
25,LilEsBella,15098105,NaN,18.68,2038,367,1.509810e+07,0.0,0.0
34,tuffyhardy,2886562144,NaN,22.82,20,50,2.886562e+09,0.0,0.0


In [7]:
######## Effective Seeds 24 ########
effective_seeds= seeds.copy()

record_num_seeds = seeds.describe().iloc[0,0]
record_num_seeds = int(record_num_seeds)
for i in range(record_num_seeds):
    k=0
    user_index = seeds.index[i]
    user_id = seeds.loc[user_index,"user_id"]
    for j in range(record_num_network):
        source_id = network.loc[j,"source_id"]
        if user_id == source_id:
            k=k+1
            #print(k)
    if k == 0:
        effective_seeds.drop([user_index],axis=0, inplace = True)


## ③. Time to iterate

In [8]:
# t: 【30-1410】 ---24小时47个半小时记录


# SImulation: features extraction + result

In [9]:
### seeds ###
def seeds_setting(seeds):
    for i in range(len(seeds)):
        index = seeds.index[i]
        network_simulation.loc[index,"SIR"] = 1
        network_simulation.loc[index,"generations"] = 0
        network_simulation.loc[index,"source_id"] = network_simulation.loc[index,"user_id"]
        network_simulation.loc[index,"seed_user_id"] = network_simulation.loc[index,"user_id"]
        network_simulation.loc[index,"time_lapsed"] = seeds.loc[index,"time_lapsed"]
        network_simulation.loc[index,"time_since_seed"] = 0
    return network_simulation

### infec_candidates ###
def infec_candidates(generation):
    generation = 
    infec_candidates = network_simulation.loc[i,"infec_candidates"]
    for j in range(len(infec_candidates)):
        infector = network_simulation.loc[i,"infec_candidates"][j]
        user_id1 = network_simulation.loc[infector,"user_id"]
        user_id2 = temp_X[temp_X['user_id'].isin([user_id1])]
        
        
infec_candidates(2)

SyntaxError: invalid syntax (<ipython-input-9-59b89593973b>, line 15)

In [ ]:
##Evaluation

##calculate the range
count = 0
for i in range(len(network_simulation)):
    if network_simulation.loc[i,'label'] ==1:
        count +=1
print('Number of infectors:',count)




In [ ]:
##important nodes(degree>?)
count = 0
for i in range(len(network)):
    if network_simulation.loc[i,'degree'] > ?:
        important_nodes.loc[i]=network[i]
for i in range(len(important_nodes)):
    j=important_nodes.iloc[i,'index']
    if important_nodes.loc[j,'label'] == network_simulation.loc[j,'label']:
        count +=1;
        
print('Accuracy of improtant nodes:',count/len(important_nodes))

In [ ]:
def loadModel(self):
    read_file = "data/" + self.model_name + ".dat"
    self.model = joblib.load(read_file)
    return self.model

model = joblib.load(path+"/xgb_model.dat")

model